In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import osmnx as ox
import matplotlib.pyplot as plt

In [12]:
tmc = pd.read_excel('../raw_data/Thailand_T19_v3.2_flat.xlsx')
tmc_gdf = gpd.GeoDataFrame(
    tmc,
    geometry=gpd.points_from_xy(tmc.LONGITUDE, tmc.LATITUDE),
    crs="EPSG:4326"
)

bangkok = ox.geocode_to_gdf("R92277", by_osmid=True)

tmc_utm = tmc_gdf.to_crs(32647)
bangkok_utm = bangkok.to_crs(32647)
poly_utm = bangkok_utm.geometry.iloc[0]

In [16]:
inside = tmc_utm[tmc_utm.within(poly_utm)]
inside.to_csv("../processed_data/bkk_loc_table.csv")